In [2]:
 from google.colab import drive
 
#  drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


# New Section

In [3]:
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LogisticRegression
import joblib
import numpy as np


In [4]:
# Load the data into a pandas dataframe

df = pd.read_csv("gdrive/MyDrive/cancer_detection/clean.csv", usecols=['customer_id', 'event', 'event_timestamp'])


<ipython-input-4-2b099c6b92cd>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("gdrive/MyDrive/cancer_detection/clean.csv", usecols=['customer_id', 'event', 'event_timestamp'])


In [5]:
df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])
df['event_date'] = df['event_timestamp'].dt.date
df = df.sort_values(['customer_id', 'event_timestamp'])
df['last_event'] = df.groupby('customer_id')['event'].shift(1)
df['last_event_date'] = df.groupby('customer_id')['event_date'].shift(1)
df['time_since_last_event'] = df['event_date'] - df['last_event_date']
df['time_since_last_event'] = df['time_since_last_event'].dt.days
df = df.dropna()
df

,customer_id,event,event_timestamp,event_date,last_event,last_event_date,time_since_last_event
729,1,new order,2023-01-13 02:00:07,2023-01-13,new order,2023-01-13,0.0
9810,1,new order,2023-01-13 02:00:07,2023-01-13,new order,2023-01-13,0.0
9813,1,new order,2023-01-13 02:00:07,2023-01-13,new order,2023-01-13,0.0
773,1,new order,2023-01-20 02:00:06,2023-01-20,new order,2023-01-13,7.0
800,1,new order,2023-01-20 02:00:06,2023-01-20,new order,2023-01-20,0.0
...,...,...,...,...,...,...,...
48743,d7daadc4cf174d81b3b83730fc5a2e0f,create customer,2022-09-26 12:54:11,2022-09-26,create customer,2022-09-26,0.0
55988,e6b1d9354c5f4f5aa87bd9cd7cd3f8fb,new order,2022-09-19 01:00:36,2022-09-19,new order,2022-09-13,6.0
48278,f6543325fc5142b69ff3a6b7410c8634,new order,2022-09-15 01:00:42,2022-09-15,new order,2022-09-15,0.0
48249,f6543325fc5142b69ff3a6b7410c8634,new order,2022-09-22 01:00:18,2022-09-22,new order,2022-09-15,7.0


In [6]:
df['time_since_last_event']

729      0.0
9810     0.0
9813     0.0
773      7.0
800      0.0
        ... 
48743    0.0
55988    6.0
48278    0.0
48249    7.0
48279    0.0
Name: time_since_last_event, Length: 108716, dtype: float64

In [7]:
# Update churned column based on time_since_last_event
df['churned'] = df['time_since_last_event'] >= 30
df

<ipython-input-7-09b9be78135e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['churned'] = df['time_since_last_event'] >= 30


,customer_id,event,event_timestamp,event_date,last_event,last_event_date,time_since_last_event,churned
729,1,new order,2023-01-13 02:00:07,2023-01-13,new order,2023-01-13,0.0,False
9810,1,new order,2023-01-13 02:00:07,2023-01-13,new order,2023-01-13,0.0,False
9813,1,new order,2023-01-13 02:00:07,2023-01-13,new order,2023-01-13,0.0,False
773,1,new order,2023-01-20 02:00:06,2023-01-20,new order,2023-01-13,7.0,False
800,1,new order,2023-01-20 02:00:06,2023-01-20,new order,2023-01-20,0.0,False
...,...,...,...,...,...,...,...,...
48743,d7daadc4cf174d81b3b83730fc5a2e0f,create customer,2022-09-26 12:54:11,2022-09-26,create customer,2022-09-26,0.0,False
55988,e6b1d9354c5f4f5aa87bd9cd7cd3f8fb,new order,2022-09-19 01:00:36,2022-09-19,new order,2022-09-13,6.0,False
48278,f6543325fc5142b69ff3a6b7410c8634,new order,2022-09-15 01:00:42,2022-09-15,new order,2022-09-15,0.0,False
48249,f6543325fc5142b69ff3a6b7410c8634,new order,2022-09-22 01:00:18,2022-09-22,new order,2022-09-15,7.0,False


In [8]:
churned_df = df[df['churned'] == False]
churned_df

,customer_id,event,event_timestamp,event_date,last_event,last_event_date,time_since_last_event,churned
729,1,new order,2023-01-13 02:00:07,2023-01-13,new order,2023-01-13,0.0,False
9810,1,new order,2023-01-13 02:00:07,2023-01-13,new order,2023-01-13,0.0,False
9813,1,new order,2023-01-13 02:00:07,2023-01-13,new order,2023-01-13,0.0,False
773,1,new order,2023-01-20 02:00:06,2023-01-20,new order,2023-01-13,7.0,False
800,1,new order,2023-01-20 02:00:06,2023-01-20,new order,2023-01-20,0.0,False
...,...,...,...,...,...,...,...,...
48743,d7daadc4cf174d81b3b83730fc5a2e0f,create customer,2022-09-26 12:54:11,2022-09-26,create customer,2022-09-26,0.0,False
55988,e6b1d9354c5f4f5aa87bd9cd7cd3f8fb,new order,2022-09-19 01:00:36,2022-09-19,new order,2022-09-13,6.0,False
48278,f6543325fc5142b69ff3a6b7410c8634,new order,2022-09-15 01:00:42,2022-09-15,new order,2022-09-15,0.0,False
48249,f6543325fc5142b69ff3a6b7410c8634,new order,2022-09-22 01:00:18,2022-09-22,new order,2022-09-15,7.0,False


In [9]:
# Build logistic regression model
X = df[['time_since_last_event']]
y = df['churned']
model = LogisticRegression()
model.fit(X, y)


LogisticRegression()

In [10]:
# Save model
joblib.dump(model, 'churn_model.joblib')

['churn_model.joblib']

In [11]:
# Load model
loaded_model = joblib.load('churn_model.joblib')

In [12]:
# # customer_id = 3481
# customer_id = 3599
# # customer_id = "f6543325fc5142b69ff3a6b7410c8634"
# # Get the most recent event date for the customer
# df_customer = df[df['customer_id'] == customer_id]
# last_event_date = df_customer['event_timestamp'].max().date()

# # Compute the time since the last event in days
# time_since_last_event = (datetime.now().date() - last_event_date).days
# new_data = pd.DataFrame({'time_since_last_event': [time_since_last_event]})
# new_data.columns = ['time_since_last_event']


# # Predict the probability of churn for the customer
# churn_prob = loaded_model.predict_proba(new_data)[:,1][0]
# churn = loaded_model.predict(new_data)
# print("Probability:", churn_prob)
# if time_since_last_event >= 30:
#     # Compute the time until churn in days
#     time_until_churn = int(np.ceil(30 / churn_prob))

#     # Compute the churn date as the most recent event date plus the time until churn
#     churn_date = last_event_date + timedelta(days=time_until_churn)

#     # Check if churn date is in the past and adjust accordingly
#     if churn_date < datetime.now().date():
#         churn_date = datetime.now().date() + timedelta(days=1)

#     print(f"Customer {customer_id} is likely to churn on {churn_date}")
# else:
#     print(f"Customer {customer_id} is not likely to churn in the near future")







## for all customers

In [48]:
# Create an empty dataframe with all unique customer IDs
final_df = pd.DataFrame({'customer_id': df['customer_id'].unique(), 'churn_date': None, 'churn_prob': None})

for i, row in final_df.iterrows():
    customer_id = row['customer_id']
    
    # Get the most recent event date for the customer
    df_customer = df[df['customer_id'] == customer_id]
    last_event_date = df_customer['event_timestamp'].max().date()

    # Compute the time since the last event in days
    time_since_last_event = (datetime.now().date() - last_event_date).days
    new_data = pd.DataFrame({'time_since_last_event': [time_since_last_event]})
    new_data.columns = ['time_since_last_event']

    # Predict the probability of churn for the customer
    churn_prob = loaded_model.predict_proba(new_data)[:,1][0]
    # Add random noise to the predicted probability
    churn_prob += np.random.normal(loc=0, scale=0.1)

    if time_since_last_event >= 30:
        # Compute the time until churn in days
        time_until_churn = int(np.ceil(30 / churn_prob))

        # Compute the churn date as the most recent event date plus the time until churn
        churn_date = last_event_date + timedelta(days=time_until_churn)

        # Check if churn date is in the past and adjust accordingly
        if churn_date < datetime.now().date():
            time_until_churn = (datetime.now().date() - last_event_date).days
            churn_date = datetime.now().date() + timedelta(days=time_until_churn)

        final_df.at[i, 'churn_date'] = churn_date

    # Store the churn probability for the customer
    final_df.at[i, 'churn_prob'] = churn_prob

print(final_df.head())


  customer_id churn_date churn_prob
0           1       None  -0.042545
1           6       None   0.122556
2           7       None    0.00077
3           8       None   0.036802
4          10       None   0.016567


In [49]:
final_df

,customer_id,churn_date,churn_prob
0,1,None,-0.042545
1,6,None,0.122556
2,7,None,0.00077
3,8,None,0.036802
4,10,None,0.016567
...,...,...,...
10078,c8305d1dba4d4b70a0772d41ddb5d60e,2023-10-28,0.987659
10079,cdf19f95c2844259bd417ec2717ea8ae,2023-10-25,1.095639
10080,d7daadc4cf174d81b3b83730fc5a2e0f,2023-10-25,0.966924
10081,e6b1d9354c5f4f5aa87bd9cd7cd3f8fb,2023-11-01,1.037774


In [50]:
final_df = final_df.append(final_df, ignore_index=True)

<ipython-input-50-fccba10629c2>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(final_df, ignore_index=True)


In [51]:
final_df.to_csv('gdrive/MyDrive/cancer_detection/CustomerchurnAndNotChern.csv', index=False)

In [52]:
final_df['churn_date'] = pd.to_datetime(final_df['churn_date'])

In [53]:
final_df

,customer_id,churn_date,churn_prob
0,1,NaT,-0.042545
1,6,NaT,0.122556
2,7,NaT,0.00077
3,8,NaT,0.036802
4,10,NaT,0.016567
...,...,...,...
20161,c8305d1dba4d4b70a0772d41ddb5d60e,2023-10-28,0.987659
20162,cdf19f95c2844259bd417ec2717ea8ae,2023-10-25,1.095639
20163,d7daadc4cf174d81b3b83730fc5a2e0f,2023-10-25,0.966924
20164,e6b1d9354c5f4f5aa87bd9cd7cd3f8fb,2023-11-01,1.037774


Only Churn and Formated Date CSV

In [54]:

final_df_churn = final_df.dropna(subset=['churn_date'])


In [55]:
# Convert churn_date to desired date format
final_df_churn['churn_date'] = final_df_churn['churn_date'].dt.strftime('%b %d, %Y')

<ipython-input-55-be6816a6699b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_churn['churn_date'] = final_df_churn['churn_date'].dt.strftime('%b %d, %Y')


In [56]:
final_df_churn

,customer_id,churn_date,churn_prob
20,28,"Jun 24, 2023",0.801844
25,33,"Jun 28, 2023",1.144014
33,41,"Jun 12, 2023",1.173019
39,47,"Jul 08, 2023",1.076952
74,86,"Jul 03, 2023",1.011566
...,...,...,...
20161,c8305d1dba4d4b70a0772d41ddb5d60e,"Oct 28, 2023",0.987659
20162,cdf19f95c2844259bd417ec2717ea8ae,"Oct 25, 2023",1.095639
20163,d7daadc4cf174d81b3b83730fc5a2e0f,"Oct 25, 2023",0.966924
20164,e6b1d9354c5f4f5aa87bd9cd7cd3f8fb,"Nov 01, 2023",1.037774


In [57]:
final_df_churn = final_df_churn.append(final_df_churn, ignore_index=True)

<ipython-input-57-41e6ba94e0f8>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df_churn = final_df_churn.append(final_df_churn, ignore_index=True)


In [58]:
final_df_churn.to_csv('gdrive/MyDrive/cancer_detection/OnlyChurn.csv', index=False)